In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import CamembertConfig, CamembertModel, AutoTokenizer, CamembertTokenizer, CamembertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import functools
from tqdm import tqdm
import gc

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
#Load 20 minutes articles
df_articles = pd.read_json('../newspaper_2.jsonl', lines=True)

In [3]:
# Create test and train dataframes
dict_labels = {'planete': 0, 'sport': 1, 'economie': 2, 'sciences': 3, 'high-tech': 4, 'politique': 5}
p_train, p_test = train_test_split(df_articles[(df_articles.category_id == 'planete')], test_size=1000, train_size=3000, random_state=42)
s_train, s_test = train_test_split(df_articles[(df_articles.category_id == 'sport')], test_size=1000, train_size=3000, random_state=42)
e_train, e_test = train_test_split(df_articles[(df_articles.category_id == 'economie')], test_size=1000, train_size=3000, random_state=42)
sc_train, sc_test = train_test_split(df_articles[(df_articles.category_id == 'sciences')], test_size=500, train_size=2000, random_state=42)
h_train, h_test = train_test_split(df_articles[(df_articles.category_id == 'high-tech')], test_size=1000, train_size=3000, random_state=42)
po_train, po_test = train_test_split(df_articles[(df_articles.category_id == 'politique')], test_size=1000, train_size=3000, random_state=42)
train_dataset = pd.concat([p_train, s_train, e_train, sc_train, h_train, po_train])[['title', 'category_id']]
train_dataset['label'] = train_dataset.apply(lambda x: dict_labels[x['category_id']], axis=1)
test_dataset = pd.concat([p_test, s_test, e_test, sc_test, h_test, po_test])[['title', 'category_id']]
test_dataset['label'] = test_dataset.apply(lambda x: dict_labels[x['category_id']], axis=1)

In [4]:
# Set gloabal parameters and tokenizer
MAX_LEN = 64
batch_size = 16
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [5]:
### Creation of the test dataset ###
# Creates list of texts and labels
text = test_dataset['title'].to_list()
labels = test_dataset['label'].to_list()

#user tokenizer to convert sentences into tokenizer
input_ids  = tokenizer(text, max_length=MAX_LEN, padding='longest', truncation=True).input_ids

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i!=1) for i in seq]  
    attention_masks.append(seq_mask)

In [6]:
# transfrom to tensor format
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# create dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [7]:
# Creates list of texts and labels
text = train_dataset['title'].to_list()
labels = train_dataset['label'].to_list()

#user tokenizer to convert sentences into tokenizer
input_ids  = tokenizer(text, max_length=MAX_LEN, padding='longest', truncation=True).input_ids

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i!=1) for i in seq]  
    attention_masks.append(seq_mask)

In [8]:
# transfrom to tensor format
train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(labels)
train_masks = torch.tensor(attention_masks)

# create dataloader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [9]:
# Load pretained model
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=6).to(device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias

In [10]:
# function to compute accuracy

def compute_accuracy(test_dataloader, model):
    total_true = 0
    total_size = 0
    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            t_data = batch[0].to(device)
            t_mask = batch[1].to(device)
            y = model(t_data,attention_mask=t_mask).logits
            result = torch.argmax(y, dim=-1).cpu().detach().numpy()
            nb_true, size = np.sum(result == np.array(batch[2])), len(result)
            total_true += nb_true
            total_size += size
            t_data.cpu()
            t_mask.cpu()
            del batch
            del y
            del t_data
            del t_mask
            gc.collect()
        
    accuracy = total_true / total_size
    return accuracy

In [11]:
# extract parameters to optimize
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

opt = torch.optim.Adam(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)


def train(classifier, data, epochs=5, log_interval=50):
    best_acc = 0
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for idx, batch in enumerate(data):
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            # Train model on batch
            opt.zero_grad()
            y_pred = classifier(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = y_pred[0]
            loss.backward()
            opt.step()
            total_loss += loss.item()
            # delete variables to keep memory on gpu
            loss.detach().cpu()
            b_input_ids.cpu()
            b_input_mask.cpu()
            b_labels.cpu()
            del(y_pred)
            del(loss)
            del(batch)
            del(b_input_ids, b_input_mask, b_labels)
            gc.collect()
            if idx % log_interval == 0 and idx > 0:
                cur_loss = total_loss / log_interval
                print(
                    "| epoch {:3d} | {:5d}/{:5d} steps | "
                    "loss {:5.5f}".format(
                        epoch, idx, len(data), cur_loss,
                    )
                )
                losses.append(cur_loss)
                total_loss = 0
        accuracy = compute_accuracy(test_dataloader, model)
        print("Test accuracy : {:1.3f}".format(accuracy))
        # Save model if better
        if accuracy > best_acc:
            torch.save(model.state_dict(), '../models/camembert_title.pt')
            best_acc = accuracy
    return classifier

In [12]:
model = train(model, train_dataloader, 3)

| epoch   0 |    50/ 1063 steps | loss 1.67313
| epoch   0 |   100/ 1063 steps | loss 1.12745
| epoch   0 |   150/ 1063 steps | loss 0.79563
| epoch   0 |   200/ 1063 steps | loss 0.64746
| epoch   0 |   250/ 1063 steps | loss 0.57306
| epoch   0 |   300/ 1063 steps | loss 0.58230
| epoch   0 |   350/ 1063 steps | loss 0.46233
| epoch   0 |   400/ 1063 steps | loss 0.53558
| epoch   0 |   450/ 1063 steps | loss 0.46556
| epoch   0 |   500/ 1063 steps | loss 0.42260
| epoch   0 |   550/ 1063 steps | loss 0.46531
| epoch   0 |   600/ 1063 steps | loss 0.45846
| epoch   0 |   650/ 1063 steps | loss 0.42746
| epoch   0 |   700/ 1063 steps | loss 0.39499
| epoch   0 |   750/ 1063 steps | loss 0.37881
| epoch   0 |   800/ 1063 steps | loss 0.37854
| epoch   0 |   850/ 1063 steps | loss 0.42017
| epoch   0 |   900/ 1063 steps | loss 0.40318
| epoch   0 |   950/ 1063 steps | loss 0.41158
| epoch   0 |  1000/ 1063 steps | loss 0.37898
| epoch   0 |  1050/ 1063 steps | loss 0.38384


100%|██████████| 344/344 [00:43<00:00,  7.93it/s]


Test accuracy : 0.871
| epoch   1 |    50/ 1063 steps | loss 0.23886
| epoch   1 |   100/ 1063 steps | loss 0.24896
| epoch   1 |   150/ 1063 steps | loss 0.23645
| epoch   1 |   200/ 1063 steps | loss 0.18058
| epoch   1 |   250/ 1063 steps | loss 0.22725
| epoch   1 |   300/ 1063 steps | loss 0.18875
| epoch   1 |   350/ 1063 steps | loss 0.23713
| epoch   1 |   400/ 1063 steps | loss 0.22972
| epoch   1 |   450/ 1063 steps | loss 0.21263
| epoch   1 |   500/ 1063 steps | loss 0.26363
| epoch   1 |   550/ 1063 steps | loss 0.20662
| epoch   1 |   600/ 1063 steps | loss 0.24101
| epoch   1 |   650/ 1063 steps | loss 0.20196
| epoch   1 |   700/ 1063 steps | loss 0.26192
| epoch   1 |   750/ 1063 steps | loss 0.23935
| epoch   1 |   800/ 1063 steps | loss 0.23952
| epoch   1 |   850/ 1063 steps | loss 0.20182
| epoch   1 |   900/ 1063 steps | loss 0.21730
| epoch   1 |   950/ 1063 steps | loss 0.25327
| epoch   1 |  1000/ 1063 steps | loss 0.29918
| epoch   1 |  1050/ 1063 steps | loss

100%|██████████| 344/344 [00:44<00:00,  7.81it/s]


Test accuracy : 0.874
| epoch   2 |    50/ 1063 steps | loss 0.14834
| epoch   2 |   100/ 1063 steps | loss 0.11662
| epoch   2 |   150/ 1063 steps | loss 0.11539
| epoch   2 |   200/ 1063 steps | loss 0.12549
| epoch   2 |   250/ 1063 steps | loss 0.10465
| epoch   2 |   300/ 1063 steps | loss 0.10881
| epoch   2 |   350/ 1063 steps | loss 0.09504
| epoch   2 |   400/ 1063 steps | loss 0.10627
| epoch   2 |   450/ 1063 steps | loss 0.09017
| epoch   2 |   500/ 1063 steps | loss 0.13100
| epoch   2 |   550/ 1063 steps | loss 0.12735
| epoch   2 |   600/ 1063 steps | loss 0.14032
| epoch   2 |   650/ 1063 steps | loss 0.12218
| epoch   2 |   700/ 1063 steps | loss 0.13462
| epoch   2 |   750/ 1063 steps | loss 0.15819
| epoch   2 |   800/ 1063 steps | loss 0.12988
| epoch   2 |   850/ 1063 steps | loss 0.12167
| epoch   2 |   900/ 1063 steps | loss 0.10256
| epoch   2 |   950/ 1063 steps | loss 0.14515
| epoch   2 |  1000/ 1063 steps | loss 0.12128
| epoch   2 |  1050/ 1063 steps | loss

100%|██████████| 344/344 [00:43<00:00,  7.83it/s]


Test accuracy : 0.874


In [43]:
# visual test
batch = next(iter(test_dataloader))
torch.argmax(model(batch[0].to(device),attention_mask=batch[1].to(device)).logits, dim=-1), batch[2]

(tensor([1, 2, 4, 2, 0, 1, 0, 5, 5, 2, 2, 2, 5, 0, 2, 1], device='cuda:0'),
 tensor([1, 2, 3, 2, 3, 4, 0, 2, 5, 2, 5, 2, 5, 0, 2, 1]))